In [3]:
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re
sys.path.append('/Users/rikhoekstra/develop/republic_clean/')

from collections import defaultdict
import pandas as pd
from collections import Counter, defaultdict
import json
import republic.model.republic_document_model as rdm

repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)
    
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [5]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es(host_type='external', timeout=60)

In [1]:
#N.B change the seed for a different selection!
seed = 15
query = {
    'function_score': {
        'query': {
            'bool': {
                'must': [
                    {'match': {'type': 'resolution'}},
                    {'range': {'metadata.session_year': {'gte': 1700, 'lte': 1797}}}
                ]
            }            
        },
        'random_score': {
            'seed': seed,
            'field': '_seq_no'
        }
    }
}

In [7]:
response = rep_es.retrieve_resolutions_by_query(query=query, size=900)

using query param
resolutions


In [8]:
len(response)

900

In [15]:
df = pd.read_csv('/Users/rikhoekstra/develop/tagging/inception_input/tag_resoluties.csv', sep='\t', index_col=0)
fls = list(df.resolutie)

In [10]:
len( [r.id for r in response if r.id not in fls])

897

In [11]:
outdir = '/Users/rikhoekstra/develop/tagging/inception_input/20230109'

In [12]:
for r in response:
    fl = r.id
    with open(os.path.join(outdir, fl +'.txt'),'w') as outfl:
        txt = '\n'.join( [p.text for p in r.paragraphs])
        outfl.write(txt)

In [17]:

ndf = pd.DataFrame([r.id for r in response], columns=['resolutie'])
df.append(ndf)

,resolutie
0,session-1709-07-11-ordinaris-num-1-resolution-66
1,session-1785-04-19-ordinaris-num-1-resolution-31
2,session-1723-02-25-ordinaris-num-1-resolution-6
3,session-1778-06-22-ordinaris-num-1-resolution-5
4,session-1717-03-26-ordinaris-num-1-resolution-6
...,...
895,session-1735-08-25-ordinaris-num-1-resolution-2
896,session-1721-03-04-ordinaris-num-1-resolution-5
897,session-1712-03-31-ordinaris-num-1-resolution-14
898,session-1792-09-12-ordinaris-num-1-resolution-4


In [65]:
df.to_csv('/Users/rikhoekstra/develop/tagging/inception_input/tag_resoluties.csv', sep='\t')